<a href="https://colab.research.google.com/github/windinherhair/fs_co_tr/blob/master/FS_%E5%85%A8%E9%83%A8_HiR_CUDA9_0_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deepfakes/faceswap (Original_HighRes 全部やる用)

## **（重要）colabの12時間&90分ルール対策**

1.新しいインスタンスを起動してから12時間経つと全てリセットされる。（GoogleDriveに学習したmodelや抽出した顔を退避させます）

2.90分アイドル状態が続くとノートの接続が外れるため、** Chromeのアドオンを使って60分置きにページをリロードする。（[Auto Refresh](https://chrome.google.com/webstore/detail/auto-refresh/ifooldnmmcmlbdennkpdnlnbgbmfalko)）**



### 事前準備

**1.[cuDNN](https://developer.nvidia.com/cudnn) 7.41 for CUDA 9.0 (cuDNN Library for Linux)をダウンロードし、GoogleDriveにアップしておく。要登録。バージョン間違える人多いので注意。**※windowsPCの場合拡張子がおかしくなるかも。拡張子をtgzに直してください。

2.体側の人の2分くらいの動画（videoA.mp4）、洗練されたBの顔セット（alignB）フォルダを用意する。（なければなるべく同じ色調、角度の顔側の人の動画10分くらい（videoB.mp4）を用意する。）※()内の名前にリネームしてください。


3.Google DriveにvideoA.mp4、alignBフォルダ、(なければvideoB.mp4）をアップロード



4.(推奨)学習済みmodelがある場合は、modelフォルダをGoogleDriveにアップロード

### その他
コメントアウト（#）をコードの前につけるとコードを呼び飛ばします。先に自分の条件に合わせてつけたり外したりしてから実行してください。**デフォルトだとBの顔セットがある人向けになってます。**
また、二回目からはDriveにalign(img)ができてるので、ffmpegとextractに(#)つけて、trainからやって下さい。

faceswap環境が整うまで30分かかります。

### **（設定）ランタイムのタイプをGPUに変更する** 

---



準備が整ったら、

**ランタイム→すべてのセルを実行**


---





In [1]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

0.00141319days (122.10sec)


**Google Driveをマウント**（※ここだけ要入力）


###  URLをクリックし、そのまま進めたら出てくるコードをコピー、枠にペーストしEnter




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


 元からインストールされているCUDA9.2を削除

In [3]:
!rm -rf /usr/local/cuda*
!apt-get purge nvidia*
!apt-get autoremove
!apt-get autoclean

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-390-dev' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-375-diagnostic' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-compute-390' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-390' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-396

CUDA9.0をインストール

In [4]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb -O cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get -o Dpkg::Options::="--force-overwrite" install cuda-9-0 cuda-drivers

--2019-01-15 11:53:29--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?GpHa4kLbx5dmGtituYySN8mNm-fObqd_scux7ISZ4ajKb0lgxhek-ftNgST8K9s0if28rWziDYE8nVbnId6UkjfQJWDg9wFVLoT9Lolkl6moJsyxqiaEwt9wSNWgOebMEdSPTTtREUxuNi6Ly_Sjs0qBAXAPhN5L2xMz5md487BlmXsUQApsXxME8rLh7PdtUSawR1qM0sAJgZnFKF8d [following]
--2019-01-15 11:53:29--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?GpHa4kLbx5dmGtituYySN8mNm-fObqd_scux7ISZ4ajKb0lgxhek-ftNgST8K9s0if28rWziDYE8nVbnId6UkjfQJWDg9wFVLoT9Lolkl6moJs

Driveからcudnn7.4をコピペして展開

In [0]:
!cp drive/'My Drive'/cudnn-9.0-linux-x64-v7.4.1.5.tgz ./

In [0]:
!tar -xf cudnn-9.0-linux-x64-v7.4.1.5.tgz

In [0]:
!cp -R cuda/include/* /usr/local/cuda-9.0/include

In [0]:
!cp -R cuda/lib64/* /usr/local/cuda-9.0/lib64

CUDAのパスを通す。

In [0]:
!echo 'export PATH=/usr/local/cuda-9.0/bin${PATH:+:${PATH}}' >> ~/.bashrc
!echo 'export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}' >> ~/.bashrc

In [0]:
!source ~/.bashrc
!ldconfig

dlibのインストール

In [11]:
!pip uninstall dlib

Uninstalling dlib-19.16.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/dlib-19.16.0-py3.6.egg-info
    /usr/local/lib/python3.6/dist-packages/dlib.cpython-36m-x86_64-linux-gnu.so
Proceed (y/n)? y
  Successfully uninstalled dlib-19.16.0


In [12]:
!apt update
!apt install -y cmake
#!git clone https://github.com/davisking/dlib.git
#!python setup.py install --yes USE_AVX_INSTRUCTIONS --yes DLIB_USE_CUDA
!apt install -y --no-install-recommends cmake && pip install dlib

Get:1 file:/var/cuda-repo-9-0-local  InRelease
Ign:1 file:/var/cuda-repo-9-0-local  InRelease
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor

In [0]:
import dlib

TFをダウングレード

In [14]:
!pip install tensorflow-gpu==1.12.0

    100% |████████████████████████████████| 281.7MB 88kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x4fca0000 @  0x7f9795da12a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


ffmpegをインストール

In [15]:
#!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.4-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


Githubのdeepfakes/faceswapをクローン

In [0]:
#!git clone https://github.com/deepfakes/faceswap.git

In [17]:
!git clone -b train_refactor https://github.com/deepfakes/faceswap.git

Cloning into 'faceswap'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 2575 (delta 1), reused 3 (delta 0), pack-reused 2560
Receiving objects: 100% (2575/2575), 173.05 MiB | 34.52 MiB/s, done.
Resolving deltas: 100% (1634/1634), done.


In [19]:
ls

Dockerfile.cpu*  INSTALL.md*  plugins/           scripts/   tools/
Dockerfile.gpu*  lib/         README.md*         setup.cfg  tools.py*
faceswap.py*     LICENSE      requirements.txt*  setup.py*  USAGE.md*


faceswapディレクトリに移動

In [18]:
cd faceswap

/content/faceswap


残りの必要なものをインストール

In [20]:
!pip install -r requirements.txt*

    100% |████████████████████████████████| 12.6MB 3.9MB/s 
    100% |████████████████████████████████| 952kB 19.4MB/s 
    100% |████████████████████████████████| 100.2MB 346kB/s 
  Running setup.py bdist_wheel for scandir ... - \ done
  Stored in directory: /root/.cache/pip/wheels/4a/ca/d7/26c3620234732f2d5b3ca86d7ccb0f59a21bd7712bffbbedc2
  Running setup.py bdist_wheel for ffmpy ... - done
  Stored in directory: /root/.cache/pip/wheels/16/28/57/96aff0d874198125b03f542d854e7ebdc61a56b09a4d49de6a
  Running setup.py bdist_wheel for nvidia-ml-py3 ... - done
  Stored in directory: /root/.cache/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19023f9e66d735c7
  Running setup.py bdist_wheel for face-recognition-models ... - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built scandir ffmpy nvidia-ml-py3 face-recognition-models
  Found existing installation: matplotlib 2.1.2




---




>  ここでやっとFaceswap環境が整う。




> ```
!python faceswap.py -h
```


> でコマンドを見れるよ。


---









In [21]:
!python faceswap.py -h

usage: faceswap.py [-h] {extract,train,convert,gui} ...

positional arguments:
  {extract,train,convert,gui}
    extract             Extract the faces from pictures
    train               This command trains the model for the two faces A and
                        B
    convert             Convert a source image to a new one with the face
                        swapped
    gui                 Launch the Faceswap Graphical User Interface

optional arguments:
  -h, --help            show this help message and exit


In [18]:
ls

Dockerfile.cpu*  INSTALL.md*  plugins/           scripts/   tools/
Dockerfile.gpu*  lib/         README.md*         setup.cfg  tools.py*
faceswap.py*     LICENSE      requirements.txt*  setup.py*  USAGE.md*


In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

## Aの顔セット準備

素材をドライブからコピー

In [0]:
#!cp /content/drive/'My Drive'/videoA.mp4 /content/faceswap/



---


ffmpegでvideoAを画像にする。(1秒あたり25フレーム)

In [0]:
#!mkdir /content/drive/'My Drive'/imgA  # 出力先ディレクトリを作成しておく

In [0]:

#!ffmpeg -i /content/faceswap/videoA.mp4 -vf fps=29.97 /content/drive/'My Drive'/imgA/imgA%06d.png



---


**Extract_A**(mtcnn)


In [0]:
#!python faceswap.py extract -i /content/drive/'My Drive'/imgA/ -o /content/drive/'My Drive'/alignA/ -D mtcnn -A fan -mp

Alignments tool で複数検知したものを別のフォルダに

In [0]:
#!python tools.py alignments -j multi-faces -a /content/drive/'My Drive'/imgA/alignments.json -o move -fc /content/drive/'My Drive'/alignA 

## →別のタブでGoogleDriveのalignA/multiple_faces_～フォルダを開いて手動で顔以外を消す

In [0]:
#input() #実行一時停止用、顔の削除が終わったら枠の中でEnterを押して下さい！

移した画像をalignAに戻す

In [0]:
#mv /content/drive/'My Drive'/alignA/multiple_faces_in_frame_*?/0/* /content/drive/'My Drive'/alignA/multiple_faces_in_frame_*?/1/* /content/drive/'My Drive'/alignA 

## Bの顔セット準備 

コメントアウト(#)外して使ってください

In [0]:
#cp /content/drive/'My Drive'/videoB.mp4 /content/faceswap/


ffmpegでvideoBを画像にする。(1秒あたり5フレーム)



In [0]:
#!mkdir /content/faceswap/imgB  # 出力先ディレクトリを作成しておく

In [0]:

#!ffmpeg -i /content/faceswap/videoB.mp4 -vf fps=5 /content/faceswap/imgB/imgB%06d.png


**Extract_B**(mtcnn)



In [0]:
#!python faceswap.py extract -i /content/faceswap/imgB/ -o /content/drive/'My Drive'/alignB/ -D mtcnn -A fan -mp

誤検知を消しやすいようにalignBを並びかえる。

In [0]:

#!python tools.py sort -i /content/drive/'My Drive'/alignB/ -fp rename -s hist



## →別のタブでGoogleDriveのalignBフォルダを開いて手動で顔以外を消す



---



※便利な**tools**もあるよ

In [0]:
#!python tools.py sort -h

In [0]:
#!python tools.py alignments -h

## **顔セットをZIPから解凍する場合**

In [22]:
!unzip /content/drive/'My Drive'/alignedHS.zip

Archive:  /content/drive/My Drive/alignedHS.zip
   creating: alignedHS/
  inflating: alignedHS/alignments.json  
  inflating: alignedHS/HSap000001_0.jpg  
  inflating: alignedHS/HSap000002_0.jpg  
  inflating: alignedHS/HSap000003_0.jpg  
  inflating: alignedHS/HSap000004_0.jpg  
  inflating: alignedHS/HSap000005_0.jpg  
  inflating: alignedHS/HSap000006_0.jpg  
  inflating: alignedHS/HSap000007_0.jpg  
  inflating: alignedHS/HSap000008_0.jpg  
  inflating: alignedHS/HSap000009_0.jpg  
  inflating: alignedHS/HSap000010_0.jpg  
  inflating: alignedHS/HSap000011_0.jpg  
  inflating: alignedHS/HSap000012_0.jpg  
  inflating: alignedHS/HSap000013_0.jpg  
  inflating: alignedHS/HSap000014_0.jpg  
  inflating: alignedHS/HSap000015_0.jpg  
  inflating: alignedHS/HSap000016_0.jpg  
  inflating: alignedHS/HSap000017_0.jpg  
  inflating: alignedHS/HSap000018_0.jpg  
  inflating: alignedHS/HSap000019_0.jpg  
  inflating: alignedHS/HSap000020_0.jpg  
  inflating: alignedHS/HSap000021_0.jpg  
  inf

In [0]:
#!unzip /content/drive/'My Drive'/faceA.zip

In [0]:
#rm -r /content/faceswap/faceA 

In [0]:
#ls /content/drive/'My Drive'/




---


## (選択)**Train**(バッチ32で100回ごとに保存)※設定はお好みで

プレビューが見れないので6時間くらいを目安にやってみてください。※事前学習済みmodelがある前提です。

コメントアウト(#)外して使ってください

In [28]:
#DFaker Model
#!python faceswap.py train -A /content/faceswap/alignedHS -B /content/faceswap/alignedHS -m /content/drive/'My Drive'/model/ -s 100 -t dfaker -bs 32

01/15/2019 06:43:45 INFO     Log level set to: INFO
Using TensorFlow backend.
01/15/2019 06:43:49 INFO     Model A Directory: /content/faceswap/alignedHS
01/15/2019 06:43:49 INFO     Model B Directory: /content/faceswap/alignedHS
01/15/2019 06:43:49 INFO     Training data directory: /content/drive/My Drive/model
01/15/2019 06:43:49 INFO     Loading data, this may take a while...
01/15/2019 06:43:49 INFO     Starting. Press 'ENTER' to stop training and save model
01/15/2019 06:43:49 INFO     Loading Model from Model_Originalhighres plugin...
01/15/2019 06:44:04 INFO     loaded model weights
01/15/2019 06:44:04 INFO     Loading Trainer from Model_Originalhighres plugin...
01/15/2019 06:44:28 INFO     saving model weights
...01/15/2019 06:44:30 INFO     done
01/15/2019 06:47:31 INFO     saving model weights
...01/15/2019 06:47:33 INFO     done
01/15/2019 06:50:34 INFO     saving model weights
...01/15/2019 06:50:37 INFO     done
01/15/2019 06:53:37 INFO     saving model weights
...01/15/2

In [0]:
!python faceswap.py train -A /content/faceswap/alignedHS -B /content/faceswap/alignedHS -m /content/drive/'My Drive'/model_vill/ -s 100 -t original-villain -bs 16

01/15/2019 12:49:59 INFO     Log level set to: INFO
Using TensorFlow backend.
01/15/2019 12:50:01 INFO     Model A Directory: /content/faceswap/alignedHS
01/15/2019 12:50:01 INFO     Model B Directory: /content/faceswap/alignedHS
01/15/2019 12:50:01 INFO     Training data directory: /content/drive/My Drive/model_vill
01/15/2019 12:50:01 INFO     ===============================================
01/15/2019 12:50:01 INFO     - Starting                                    -
01/15/2019 12:50:01 INFO     - Press 'ENTER' to save and quit              -
01/15/2019 12:50:01 INFO     - Press 'S' to save model weights immediately -
01/15/2019 12:50:01 INFO     ===============================================
01/15/2019 12:50:02 INFO     Loading data, this may take a while...
01/15/2019 12:50:02 INFO     Loading Model from Original_Villain plugin...
01/15/2019 12:50:10 INFO     Updating config at: '/content/faceswap/plugins/train/config.ini'
01/15/2019 12:50:10 INFO     Loading config: '/content/face

In [0]:
#!python faceswap.py train -A /content/drive/'My Drive'/alignB/ -B /content/drive/'My Drive'/alignB/ -m /content/drive/'My Drive'/model/ -s 100 -t OriginalHighRes -bs 32
#BとBの事前学習

In [0]:
#!python faceswap.py train -A /content/faceswap/faceA -B /content/faceswap/aligned -m /content/drive/'My Drive'/model/ -s 100 -t OriginalHighRes -bs 32
#AとBの通常学習

In [23]:
#オプションはお好みで
!python faceswap.py train -h

usage: faceswap.py train [-h] [-L {INFO,VERBOSE,DEBUG,TRACE}] [-A INPUT_A]
                         [-B INPUT_B] [-m MODEL_DIR] [-s SAVE_INTERVAL]
                         [-t {dfaker,dfl-h128,iae,original,original-hires,original-villain}]
                         [-bs BATCH_SIZE] [-it ITERATIONS] [-g GPUS] [-p] [-w]
                         [-ag] [-tia TIMELAPSE_INPUT_A]
                         [-tib TIMELAPSE_INPUT_B] [-to TIMELAPSE_OUTPUT]

This command trains the model for the two faces A and B

optional arguments:
  -h, --help            show this help message and exit
  -L {INFO,VERBOSE,DEBUG,TRACE}, --loglevel {INFO,VERBOSE,DEBUG,TRACE}
                        Log level. Stick with INFO or VERBOSE unless you need
                        to file an error report. Be careful with TRACE as it
                        will generate a lot of data
  -A INPUT_A, --input-A INPUT_A
                        Input directory. A directory containing training
                        images for 



---


**Convert**



In [0]:
#!python faceswap.py convert -i /content/drive/'My Drive'/imgA/ -o /content/drive/'My Drive'/mergeAz -al /content/drive/'My Drive'/imgA/alignments.json -m /content/drive/'My Drive'/model/ -a /content/drive/'My Drive'/alignA/ -t OriginalHighRes -c Masked -b 30 -M facehull

In [19]:
#オプションはお好みで
#!python faceswap.py convert -h

Traceback (most recent call last):
  File "faceswap.py", line 5, in <module>
    import lib.cli as cli
  File "/content/faceswap/lib/cli.py", line 12, in <module>
    from lib.utils import safe_shutdown
  File "/content/faceswap/lib/utils.py", line 16, in <module>
    import dlib
ImportError: libcublas.so.9.2: cannot open shared object file: No such file or directory




---

ffmpegで画像から動画（merged_videoA.mp4）に変換

In [0]:
#!ffmpeg -framerate 29.97 -i /content/drive/'My Drive'/mergeA/imgA%06d.png -vcodec libx264 -pix_fmt yuv420p -r 29.97 /content/drive/'My Drive'/merged_videoA1.mp4

In [0]:
#ls /content/drive/'My Drive'/mergeA/

In [0]:
#cp -u /content/drive/'My Drive'/imgA/*.png /content/drive/'My Drive'/mergeA/

# 終了～